# Practica #6

## Nombre: Christian Zorrilla Gonzalez

## Matrícula: 21-SISN-2-070

## Fecha de entrega: 18 de Octubre de 2023

## Frases e imágenes

Utilizando una interfaz Gradio, crear una aplicación que reciba una entrada de texto y, al ejecutarlo, devuelva una frase relacionada y una imagen.

Ejemplo: si introducimos "guerra", debe mostrar una imagen relacionada con la guerra y una frase sobre este tema.

## Traductor

Utilizando una interfaz Gradio, crear una aplicación que reciba una entrada de texto y lo traduzca a uno de los idiomas disponibles en una lista. La traducción debe realizarse sin agregar nada más. La lista debe contener al menos 5 idiomas.

Ejemplo: si la entrada es "Watashi wa michi to shinri to inochi desu" y se selecciona el idioma español, debe devolver únicamente: "Yo soy el camino, la verdad y la vida", sin agregar nada más."

## Traductor


In [ ]:
import gradio as gr
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import torch

# this model was loaded from https://hf.co/models
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M")
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M")
device = 0 if torch.cuda.is_available() else -1
LANGS = ["ace_Arab", "eng_Latn", "fra_Latn", "spa_Latn"]

def translate(text, src_lang, tgt_lang):
    """
    Translate the text from source lang to target lang
    """
    translation_pipeline = pipeline("translation", model=model, tokenizer=tokenizer, src_lang=src_lang, tgt_lang=tgt_lang, max_length=400, device=device)
    result = translation_pipeline(text)
    return result[0]['translation_text']

demo = gr.Interface(
    fn=translate,
    inputs=[
        gr.components.Textbox(label="Text"),
        gr.components.Dropdown(label="Source Language", choices=LANGS),
        gr.components.Dropdown(label="Target Language", choices=LANGS),
    ],
    outputs=["text"],
    examples=[["Building a translation demo with Gradio is so easy!", "eng_Latn", "spa_Latn"]],
    cache_examples=False,
    title="Translation Demo",
    description="This demo is a simplified version of the original [NLLB-Translator](https://huggingface.co/spaces/Narrativaai/NLLB-Translator) space"
)

demo.launch()

## Imagenes

In [ ]:
import gradio as gr
import requests
from PIL import Image
from io import BytesIO


def obtener_imagen(palabra_clave):
    api_key = "e60807c3ce9c46e1a0e1eedc5f2a7dc8"
    endpoint = "https://api.cognitive.microsoft.com/bing/v7.0/images/search"

    try:
        headers = {"Ocp-Apim-Subscription-Key": api_key}
        params = {"q": palabra_clave, "count": 1}
        response = requests.get(endpoint, headers=headers, params=params)
        data = response.json()
        imagen_url = data["value"][0]["contentUrl"]
        imagen_resp = requests.get(imagen_url)
        imagen = Image.open(BytesIO(imagen_resp.content))
    except Exception as e:
        print(f"Error fetching image: {e}")
        imagen = None

    return imagen

def generar_frase(palabra_clave):
    frase = f"Una imagen relacionada a '{palabra_clave}'"
    return frase

interfaz = gr.Interface(
    fn=lambda texto: (obtener_imagen(texto), generar_frase(texto)),
    inputs="text",
    outputs=["image", "text"],
    layout="vertical",
    examples=[["guerra"]]
)

interfaz.launch()